In [ ]:
import numpy as np
import pickle as pk
import pandas as pd
import json
import re
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Load the JSON data
with open('project_data.json') as file:
    data = json.load(file)

In [ ]:
# Extract patterns, labels, and responses from the JSON data
training_patterns = []
training_labels = []
responses = {}

for intent in data['intents']:
    patterns = intent['patterns']
    tag = intent['tag']
    response = intent['responses']

    training_patterns.extend(patterns)
    training_labels.extend([tag] * len(patterns))
    responses[tag] = response

In [ ]:
# Clean and preprocess the text patterns using regular expressions and lemmatization
def clean_text(sentences):
    result = []
    lemmatizer = WordNetLemmatizer()
    for sentence in sentences:
        # Convert to lowercase
        sentence = sentence.lower()
        # Tokenize the sentence
        words = word_tokenize(sentence)
        # Lemmatize each word
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        # Join the lemmatized words back into a sentence
        sentence = ' '.join(lemmatized_words)
        # Remove non-alphanumeric characters and extra whitespaces
        sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)
        # Remove extra whitespaces
        sentence = re.sub(r'\s+', ' ', sentence)
        result.append(sentence.strip())
    return result


In [ ]:
# Clean the training patterns
training_patterns = clean_text(training_patterns)

In [ ]:
# Encode the training labels
label_encoder = LabelEncoder()
training_labels = label_encoder.fit_transform(training_labels)
training_labels = to_categorical(training_labels)


In [ ]:
# Tokenize the training patterns
tokenizer = Tokenizer(num_words=500, oov_token='<OOV>')
tokenizer.fit_on_texts(training_patterns)
sequences = tokenizer.texts_to_sequences(training_patterns)

In [ ]:
# Pad sequences to ensure uniform length
max_sequence_length = 32
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_sequence_length)

In [ ]:
len(tokenizer.word_index)+1

432

In [ ]:
# Define and compile the model with LSTM layer
model = Sequential()
model.add(Embedding(input_dim=432, output_dim=32))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          13824     
                                                                 
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 127)               32639     
                                                                 
Total params: 161919 (632.50 KB)
Trainable params: 161919 (632.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
history = model.fit(np.array(padded_sequences), training_labels, epochs=50, batch_size=5, verbose=1)

Epoch 1/50
214/214 [==============================] - 18s 59ms/step - loss: 4.7472 - accuracy: 0.0112
Epoch 2/50
214/214 [==============================] - 12s 57ms/step - loss: 4.3125 - accuracy: 0.0328
Epoch 3/50
214/214 [==============================] - 12s 58ms/step - loss: 3.9890 - accuracy: 0.0609
Epoch 4/50
214/214 [==============================] - 12s 57ms/step - loss: 3.6921 - accuracy: 0.0937
Epoch 5/50
214/214 [==============================] - 12s 58ms/step - loss: 3.2288 - accuracy: 0.1621
Epoch 6/50
214/214 [==============================] - 14s 65ms/step - loss: 2.6875 - accuracy: 0.2793
Epoch 7/50
214/214 [==============================] - 13s 59ms/step - loss: 2.2018 - accuracy: 0.3955
Epoch 8/50
214/214 [==============================] - 13s 61ms/step - loss: 1.8825 - accuracy: 0.4789
Epoch 9/50
214/214 [==============================] - 13s 62ms/step - loss: 1.6183 - accuracy: 0.5408
Epoch 10/50
214/214 [==============================] - 12s 58ms/step - loss: 1.365

In [ ]:
model.save('model.h5') # save model
from tensorflow.keras.models import load_model
model=load_model('model.h5') # load model

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Function to preprocess input text using regular expressions and lemmatization
def preprocess_input(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize the sentence
    words = word_tokenize(text)
    # Lemmatize each word
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # Join the lemmatized words back into a sentence
    text = ' '.join(lemmatized_words)
    # Remove non-alphanumeric characters and extra whitespaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
# Function to predict intent
def predict_intent(input_text):
    input_text = preprocess_input(input_text)
    input_sequence = pad_sequences(tokenizer.texts_to_sequences([input_text]), maxlen=max_sequence_length)
    result_index = np.argmax(model.predict(np.array(input_sequence), verbose=0))
    predicted_intent = label_encoder.classes_[result_index]
    return predicted_intent

In [ ]:
# Function to get response
def get_response(predicted_intent):
    for intent in data['intents']:
        if intent['tag'] == predicted_intent:
            responses = intent['responses']
            if isinstance(responses, list):
                return np.random.choice(responses)
            else:
                return responses

In [ ]:
#save encoder
pk.dump(label_encoder,open("label_encoder.pkl","wb"))

In [ ]:
#save tokinizer
pk.dump(tokenizer,open("tokenizer.pkl","wb"))

In [ ]:

# Function for user chat
def user_chat():
    while True:
        print("User: ", end="")
        user_input = input()

        if user_input.lower() == 'quit':
            break

        predicted_intent = predict_intent(user_input)
        response = get_response(predicted_intent)
        print('ChatBot:', response)

# Start chatting
user_chat()


User: quit
